In [5]:
import pandas as pd
import re
import os
import numpy as np
project_path = os.path.abspath(os.path.relpath('../../../../', os.getcwd()))
import lightning as L
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

In [6]:
class CHBDependentDataset(Dataset):
    def __init__(self,data_dir:str,patient_id:int,stage:str):
        self.data_dir = data_dir
        self.interictal_path=os.path.join(self.data_dir,os.path.join('chb'+str(patient_id).zfill(2)+"_interictal.npz"))
        self.preictal_path=os.path.join(self.data_dir,os.path.join('chb'+str(patient_id).zfill(2)+"_preictal.npz"))
        self.interictal = np.load(self.interictal_path)['data']
        self.preictal = np.load(self.preictal_path)['data']
        if stage == 'train':
            self.X = np.concatenate((self.interictal[:int(0.8*len(self.interictal))],self.preictal[:int(0.8*len(self.preictal))]))
            self.y = np.concatenate((np.zeros(int(0.8*len(self.interictal))),np.ones(int(0.8*len(self.preictal)))))
        elif stage == 'val':
            self.X = np.concatenate((self.interictal[int(0.8*len(self.interictal)):],self.preictal[int(0.8*len(self.preictal)):]))
            self.y = np.concatenate((np.zeros(len(self.interictal)-int(0.8*len(self.interictal))),np.ones(len(self.preictal)-int(0.8*len(self.preictal)))))
        
        else:
            raise ValueError("Invalid stage")


    def __getitem__(self,index):
        return self.X[index],self.y[index]

    def __len__(self):
        return len(self.X)
        
        
        

In [7]:



class CHBDependentDM(L.LightningDataModule):
    def __init__(self, data_dir: str ,patient_id:int,batch_size:int=32):
        super().__init__()
        self.trainset = None
        self.valset = None
        self.testset = None
        self.batch_size = batch_size
        self.data_dir = data_dir
        self.patient_id = patient_id
        

    def prepare_data(self):
        pass

    def setup(self, stage: str):
        if stage == 'fit':
            self.trainset = CHBDependentDataset(self.data_dir,self.patient_id,stage='train')
            self.valset = CHBDependentDataset(self.data_dir,self.patient_id,stage='val')
    
        else:
            raise ValueError("Invalid stage")
        
        
    def train_dataloader(self):
        return DataLoader(self.trainset, batch_size=self.batch_size,shuffle=True,pin_memory=True)

    def val_dataloader(self):
        return DataLoader(self.valset, batch_size=self.batch_size,shuffle=False,pin_memory=True)

    def test_dataloader(self):
        return None

    def predict_dataloader(self):
        return None

In [8]:
dm = CHBDependentDM(data_dir=os.path.join(project_path,os.path.join(project_path,'BilinearNetwork/Data/PreprocessedData/CHB-MIT/STFT')),patient_id=1)
dm.prepare_data()
dm.setup(stage="fit")

In [9]:

for x,y in dm.val_dataloader():
    print(x.shape)
    
    print(y.shape)
    break